In [1]:
import heapq
import numpy as np

class Puzzle:
    def __init__(self, initial, goal, heuristic):
        self.initial = initial
        self.goal = goal
        self.heuristic = heuristic

    def get_neighbors(self, state):
        neighbors = []
        empty_index = state.index(0)
        row, col = divmod(empty_index, 3)
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for dr, dc in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_index = new_row * 3 + new_col
                new_state = list(state)
                new_state[empty_index], new_state[new_index] = new_state[new_index], new_state[empty_index]
                neighbors.append(tuple(new_state))
        
        return neighbors

    def misplaced_tiles(self, state):
        return sum(1 for i in range(9) if state[i] != 0 and state[i] != self.goal[i])
    
    def manhattan_distance(self, state):
        distance = 0
        for i in range(9):
            if state[i] != 0:
                goal_index = self.goal.index(state[i])
                row1, col1 = divmod(i, 3)
                row2, col2 = divmod(goal_index, 3)
                distance += abs(row1 - row2) + abs(col1 - col2)
        return distance
    
    def a_star(self):
        frontier = []
        heapq.heappush(frontier, (0, self.initial))
        came_from = {self.initial: None}
        cost_so_far = {self.initial: 0}
        nodes_explored = 0

        while frontier:
            _, current = heapq.heappop(frontier)
            nodes_explored += 1

            if current == self.goal:
                return self.reconstruct_path(came_from), nodes_explored

            for neighbor in self.get_neighbors(current):
                new_cost = cost_so_far[current] + 1
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    priority = new_cost + (self.misplaced_tiles(neighbor) if self.heuristic == 'H1' else self.manhattan_distance(neighbor))
                    heapq.heappush(frontier, (priority, neighbor))
                    came_from[neighbor] = current
        
        return None, nodes_explored  # No solution

    def reconstruct_path(self, came_from):
        path = []
        current = self.goal
        while current:
            path.append(current)
            current = came_from[current]
        return path[::-1]

# Example puzzle states
initial_state = (1, 2, 3, 4, 5, 6, 7, 0, 8)
goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)

# Run A* with both heuristics
puzzle_h1 = Puzzle(initial_state, goal_state, heuristic='H1')
solution_h1, nodes_h1 = puzzle_h1.a_star()

puzzle_h2 = Puzzle(initial_state, goal_state, heuristic='H2')
solution_h2, nodes_h2 = puzzle_h2.a_star()

# Output results
print(f"H1 (Misplaced Tiles) - Nodes Explored: {nodes_h1}, Solution Depth: {len(solution_h1) - 1}")
print(f"H2 (Manhattan Distance) - Nodes Explored: {nodes_h2}, Solution Depth: {len(solution_h2) - 1}")


H1 (Misplaced Tiles) - Nodes Explored: 2, Solution Depth: 1
H2 (Manhattan Distance) - Nodes Explored: 2, Solution Depth: 1
